In [ ]:
""" MODELO PRE ENTRENADO SwinViT Small
Dataset: Patch Camelyon, de tensorflow preparado para HuggingFace
Fuente del modelo: HuggingFace
Modelo: microsoft/swin-small-patch4-window7-224
"""

In [ ]:
# Conectar con Google Drivee()
from google.colab import drive
drive.mount('/content/drive')
base_folder = "/content/drive/MyDrive/00 VIU/10 TFM"


In [ ]:
# librerías
!pip install -q datasets evaluate keras_cv
!pip install --upgrade transformers
!pip install -q tensorflow
!pip install -q tf-keras


In [ ]:
import numpy as np
import transformers
import evaluate
from transformers import AutoModelForImageClassification, AutoImageProcessor,TrainingArguments, Trainer
from datasets import Dataset as HuggingFaceDataset, Features
from PIL import Image as PILImage
import torch

In [ ]:
# importar credenciales de Hugging face
from google.colab import userdata
import os
from huggingface_hub import login

# Accede al token almacenado en los secretos de Colab
hf_token = userdata.get('HF_TOKEN')

## Cargar el dataset patch camelyon de hugging face

In [ ]:
train_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/train 12k")

val_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/val 12k")

## Preprocesar los datos

In [ ]:
modelo = "microsoft/swin-small-patch4-window7-224"
batch_size = 32

# instanciar el image processing
image_processor = AutoImageProcessor.from_pretrained(modelo,do_rescale=True,use_fast=True)

In [ ]:
def preprocess_hf_dataset(examples):
    images_pil = [img for img in examples["image"]]
    inputs = image_processor(images=images_pil, return_tensors="pt")

    # inputs["pixel_values"] es un solo tensor PyTorch de forma (batch_size_interno, C, H, W)
    # Debemos dividirlo en una lista de tensores individuales (C, H, W), uno por cada imagen
    # para que el Dataset de Hugging Face lo almacene correctamente por ejemplo.
    # .unbind(0) divide el tensor a lo largo de la dimensión 0 (batch_size)
    pixel_values_list = list(inputs["pixel_values"].unbind(0))

    # `examples["label"]` es una lista de Python `int`s cuando `batched=True`
    labels_list = examples["label"]

    return {"pixel_values": pixel_values_list, "labels": labels_list}

In [ ]:
train_hf_ds = train_hf_ds.map(preprocess_hf_dataset, batched=True, remove_columns=["image"],load_from_cache_file=False)
val_hf_ds = val_hf_ds.map(preprocess_hf_dataset, batched=True, remove_columns=["image"],load_from_cache_file=False)

In [ ]:
num_labels = 2
model = AutoModelForImageClassification.from_pretrained(
    modelo,
    num_labels=num_labels,
    ignore_mismatched_sizes = True
)

In [ ]:
print(model)

In [ ]:
print(model.config)

In [ ]:
print("Cantidad de parámetros del modelo: ",(sum(p.numel() for p in model.parameters())//1e5)/10, "M")  # total parámetros


In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    references = p.label_ids
    metrics_result = metric.compute(predictions=predictions, references=references)
    print(f"Metric computation result: {metrics_result}")
    return {"eval_accuracy": metrics_result["accuracy"]}

In [ ]:
# establecer los parámetros del entrenamiento
args = TrainingArguments(
    remove_unused_columns=False,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    report_to="none",
    output_dir=base_folder+"/Modelos entrenados/Swin v1 Small"
)

### Collator

In [ ]:
import torch

def custom_image_collator(batch):
    processed_pixel_values_for_batch = []
    for example in batch:
        current_pixel_values = example["pixel_values"] # Esto es una <class 'list'>
        if isinstance(current_pixel_values, list):
            temp_tensor = None
            if len(current_pixel_values) > 0:
                if torch.is_tensor(current_pixel_values[0]):
                    if len(current_pixel_values) == 3 and all(torch.is_tensor(x) for x in current_pixel_values):
                        temp_tensor = torch.stack(current_pixel_values) # stackea canales: (C, H, W)
                    elif len(current_pixel_values) == 1 and torch.is_tensor(current_pixel_values[0]):
                        temp_tensor = current_pixel_values[0]
                    else:
                        raise ValueError(f"Unexpected list of Tensors structure for pixel_values: {current_pixel_values[0].shape if len(current_pixel_values) > 0 else 'empty list'}")
                elif isinstance(current_pixel_values[0], list):
                    temp_tensor = torch.tensor(current_pixel_values, dtype=torch.float32) # Convertir la lista anidada a tensor
                else:
                    raise ValueError(f"Unexpected element type in pixel_values list: {type(current_pixel_values[0])}. Expected Tensor or list.")
            else:
                raise ValueError("pixel_values list is empty.")
        elif torch.is_tensor(current_pixel_values):
            temp_tensor = current_pixel_values
        else:
            raise ValueError(f"Unexpected main type for pixel_values: {type(current_pixel_values)}. Expected list or Tensor.")

        processed_pixel_values_for_batch.append(temp_tensor)

    stacked_pixel_values = torch.stack(processed_pixel_values_for_batch)


    labels_clean = []
    for example in batch:
        if isinstance(example["labels"], torch.Tensor):
            labels_clean.append(example["labels"].item() if example["labels"].numel() == 1 else example["labels"].tolist()[0])
        else:
            labels_clean.append(example["labels"])
    stacked_labels = torch.tensor(labels_clean, dtype=torch.long)

    return {"pixel_values": stacked_pixel_values, "labels": stacked_labels}

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_hf_ds,
    eval_dataset=val_hf_ds,
    compute_metrics=compute_metrics,
    data_collator=custom_image_collator,
    tokenizer=image_processor
)

In [ ]:
%%time
print("\n--- Iniciando entrenamiento ---")
train_results = trainer.train()

In [ ]:
trainer.save_model(base_folder+"/Modelos entrenados/Swin Small v1")

In [ ]:
# EVALUAR EL MODELO

# Cargar el dataset de test 32k
test_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/test")

# Preprocesar el dataset de test
test_hf_ds = test_hf_ds.map(preprocess_hf_dataset, batched=True, remove_columns=["image"], load_from_cache_file=False)

# Evaluar el modelo con el dataset de test


In [ ]:
%%time
test_results = trainer.evaluate(test_hf_ds)

print("\n--- Resultados de la evaluación en el dataset de test ---")
print(test_results)


In [ ]:
print(model.config)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")